In [2]:
text = ""

with open("./input.txt", "r") as file:
	text = file.read()
	

print("Total chars: ", len(text))

Total chars:  1115394


In [3]:
vocabulary_list = sorted(list(set(text)))
vocab_size = len(vocabulary_list)

print("Vocabulary: ", "".join(vocabulary_list))
print("Lenght of vocabulary: ", vocab_size)

Vocabulary:  
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Lenght of vocabulary:  65


In [4]:
stoi = { char: i for i, char in enumerate(vocabulary_list) }
itos = { i: char for i, char in enumerate(vocabulary_list) }

encode = lambda s: [stoi[char] for char in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("test encoding"))
print(decode(encode("test encoding")))

[58, 43, 57, 58, 1, 43, 52, 41, 53, 42, 47, 52, 45]
test encoding


In [5]:
import torch

In [6]:
tensor = torch.tensor(encode(text), dtype=torch.long)
print(tensor.shape, tensor.dtype)
print(tensor[:1000])

n = int(0.9 * len(tensor))
train_data, val_data = tensor[:n], tensor[n:]

block_size = 8
print(train_data[:block_size+1])


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
	context = x[:t + 1]
	target = y[t]

	print(f"when input is {context} then target is {target}")

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
batch_size = 4 # how many independent sequences will we process in parallel
block_size = 8 # what is the maximum context length for predictions

def get_batch(split):
	data = train_data if split == "train" else val_data
	ix = torch.randint(len(data) - block_size, (batch_size,))
	print("ix", ix)

	x = torch.stack([data[i:i + block_size] for i in ix])
	y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])

	return x, y

xb, yb = get_batch("train")
print("inputs:")

print(xb.shape)
print(xb)

print("targets:")
print(yb.shape)
print(yb)

print('------')

for b in range(batch_size):
	for t in range(block_size):
		context = xb[b, :t + 1]
		target = yb[b, t]

		print(f"when input is {context} then target is {target}")

ix tensor([ 95616, 721361, 959083, 585286])
inputs:
torch.Size([4, 8])
tensor([[59, 56,  1, 57, 46, 39, 49, 43],
        [ 0, 13, 57,  1, 51, 47, 52, 43],
        [56,  1, 41, 53, 52, 58, 56, 39],
        [33, 25, 14, 17, 30, 24, 13, 26]])
targets:
torch.Size([4, 8])
tensor([[56,  1, 57, 46, 39, 49, 43,  1],
        [13, 57,  1, 51, 47, 52, 43,  6],
        [ 1, 41, 53, 52, 58, 56, 39, 41],
        [25, 14, 17, 30, 24, 13, 26, 16]])
------
when input is tensor([59]) then target is 56
when input is tensor([59, 56]) then target is 1
when input is tensor([59, 56,  1]) then target is 57
when input is tensor([59, 56,  1, 57]) then target is 46
when input is tensor([59, 56,  1, 57, 46]) then target is 39
when input is tensor([59, 56,  1, 57, 46, 39]) then target is 49
when input is tensor([59, 56,  1, 57, 46, 39, 49]) then target is 43
when input is tensor([59, 56,  1, 57, 46, 39, 49, 43]) then target is 1
when input is tensor([0]) then target is 13
when input is tensor([ 0, 13]) then target

In [8]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
        
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indeces in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)

            logits = logits[:, -1, :]  # becomes (B, C)
            probs = F.softmax(logits, dim=1)  # (B, C)

            # sample from the distribution
            idx_next  = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
generated_sequence = m.generate(idx, max_new_tokens=100)
print(decode(generated_sequence[0].tolist()))

torch.Size([32, 65])
tensor(4.6174, grad_fn=<NllLossBackward0>)

IxXFQLdOcP&qOq:
ntm
U:ywpZZI:zSy CXWhPaAPeYb?CXLzOCtp rwCM,kIhC:PaBVSvswp:  rCJRSqDa:dh&ShMucF.M'vsM


In [9]:
total_params = sum(p.numel() for p in m.parameters())
print(f'Total number of parameters: {total_params}')

Total number of parameters: 4225


In [10]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32

for step in range(10000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)

    print(loss.item())
    
    loss.backward()
    optimizer.step()



ix tensor([316701, 434196,  23978, 248303, 109686, 270514, 579848, 392338, 553744,
        102680, 611365, 300595, 192532, 125948, 331506, 385078, 909075, 587588,
        524675, 340448, 211058, 397645, 105255, 805172,  95889, 834570, 958447,
         16765, 390573, 623105, 310102, 283102])
2.475893497467041
ix tensor([909081, 798742, 857375,   1198, 488415, 781312, 985186, 894120, 926060,
        362605, 352798, 263816,  15226,  61060, 473949, 449411, 182101, 333626,
        842406,  60473, 471904, 510526, 385592, 699183, 351473,  48339, 700849,
        822347, 810122, 959524,  81012, 944212])
2.5547871589660645
ix tensor([314288, 487072, 110305, 133296, 525947, 448983, 896486, 415587, 109604,
         98623, 915628, 784391, 222845, 949076, 865120, 885178, 647346, 684890,
        222744, 557775, 152484, 609120, 242595, 155714, 548156,   8980, 648321,
        213288, 917574, 228152, 845222, 797639])
2.395782709121704
ix tensor([637320, 587383, 828189, 674534, 709393, 845845, 661780, 85

In [17]:
idx = torch.zeros((1, 1), dtype=torch.long)
generated_sequence = m.generate(idx, max_new_tokens=400)
print(decode(generated_sequence[0].tolist()))


NETORD be aner hivetr llyo h or SThaloubr I bemed bl a coved! re a artowharinchin hin:
S:
SSAnd u theshethemagome werg hene s, d t A:

Thithelle sere.

Sim; foule hindere knt plow se
YOu vidy d thaithe seshipruny mis thomy ace t wnve
Kiore
SAnd s

Whee igad me, weicats a, bemeshacor-ldes,
R: dand donshit chasiser isullotolousisentes ls. d t as qu,

Peearr t r Bes wargou s se.
PO:
ag t f, t
Lugh t 
